In [8]:
import pandas as pd

greats_df = pd.read_csv("./data/career_stats_greats.csv")
szn_86_df = pd.read_csv("./data/allplayers_85_86_szn.csv")
szn_23_df = pd.read_csv("./data/allplayers_22_23_szn.csv")

In [10]:
szn_86_df

,Rk,Player,Age,Tm,Pos,GP,G,A,PTS,+/-,...,SH,GW,EV.1,PP.1,SH.1,S,S%,TOI,ATOI,-9999
0,1,Keith Acton,27,MNS,C,79,26,32,58,-12,...,2,2,23,7,2,169,15.4,NaN,NaN,actonke01
1,2,Greg Adams,25,WSH,LW,78,18,38,56,24,...,0,2,27,11,0,150,12.0,NaN,NaN,adamsgr01
2,3,Greg Adams,22,NJD,C,78,35,42,77,-7,...,0,2,31,11,0,202,17.3,NaN,NaN,adamsgr02
3,4,Mike Allison,24,NYR,LW,28,2,13,15,4,...,0,0,13,0,0,26,7.7,NaN,NaN,allismi01
4,5,Glenn Anderson*,25,EDM,RW,72,54,48,102,38,...,2,9,33,15,0,242,22.3,NaN,NaN,andergl01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
725,615,Warren Young,30,DET,LW,79,22,24,46,-33,...,0,1,16,8,0,92,23.9,NaN,NaN,youngwa01
726,616,Steve Yzerman*,20,DET,C,51,14,28,42,-22,...,0,3,13,15,0,132,10.6,NaN,NaN,yzermst01
727,617,Peter Zezel,20,PHI,C,79,17,37,54,27,...,0,4,24,11,2,146,11.6,NaN,NaN,zezelpe01
728,618,Rick Zombo,22,DET,D,14,0,1,1,-10,...,0,0,1,0,0,8,0.0,NaN,NaN,zombori01
